In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input
from keras.models import Model, Sequential  

df_train = pd.read_csv("sales_train_validation.csv")
#df_sell = pd.read_csv("./storage/Kaggle_M5_Luke/sell_prices.csv")
#df_calendar = pd.read_csv("./storage/Kaggle_M5_Luke/calendar.csv")
#df_ss = pd.read_csv("./storage/Kaggle_M5_Luke/sample_submission_m5.csv")


Using TensorFlow backend.


In [2]:
days = range(1,1913+1) 
time_series_cols = [] 
for i in days: 
    time_series_cols.append('d_'+str(i)) 
time_series_cols[:10]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']

In [3]:
time_series_data = df_train[time_series_cols] 
time_series_data

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [4]:
time_series_data = np.asarray(time_series_data).astype('float32')
print(time_series_data.shape)

(30490, 1913)


In [12]:
#mean = [] # list of mean values 
#sd = [] # list of standard deviations 

# normalize row by row 
#for i in range(len(time_series_data)):  
#    mean.append(np.mean(time_series_data[i])) 
#    sd.append(np.std(time_series_data[i])) 
#    time_series_data[i] = (time_series_data[i]-np.mean(time_series_data[i]))/np.std(time_series_data[i]) 

In [5]:
time_series_data 

array([[0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 6., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 3., 1., 3.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
def build_model(time_steps,features): 
    model = Sequential() 
    model.add(LSTM(128,input_shape=(time_steps,features),activation='relu')) 
    model.add(Dense(30490))   
    return model 
#model = build_model(7,30490)
#model.summary()

In [7]:
# let's now code for the general case: for all the 28 days 
def split_xy2(dataset, time_steps):
    x,y = list(), list() 
    for i in range(0,dataset.shape[1]-time_steps): 
        x.append(dataset[:,i:i+time_steps]) 
        y.append(dataset[:,i+time_steps]) 
    return np.array(x), np.array(y) 

predictions = [] 

for i in range(1,29): 
    print("day {}".format(i)) 
    dataset = time_series_data[:,::i] 
    x_train, y_train = split_xy2(dataset,7) 
    x_pred = dataset[:,dataset.shape[1]-7:]  
    x_train = x_train.reshape(x_train.shape[0],7,30490) 
    x_pred = x_pred.reshape(1,7,30490)  
    model = build_model(7,30490) 
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
    model.fit(x_train,y_train,epochs=50) 
    pred = model.predict(x_pred) 
    print(pred[0]) # for checking purposes 
    predictions.append(pred) 
    #time_series_data = np.append(time_series_data,pred.T,axis=1)  

#for i in range(1,29): 
#    print("day {}".format(i)) 
#    dataset = time_series_data[:,::i] 
#    x_train, y_train = split_xy2(dataset,7) 
#    x_pred = dataset[:,dataset.shape[1]-7:] 
#    x_train = x_train.reshape(x_train.shape[0],7,30490) 
#    x_pred = x_pred.reshape(1,7,30490) 
#    model = build_model(7,30490) 
#    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
#    model.fit(x_train,y_train,epochs=500) 
#    pred = model.predict(x_pred) 
#    predictions.append(pred) 
    


day 1
Epoch 1/50
1906/1906 [==============================] - 3s 2ms/step - loss: 10.3473 - mse: 10.3473
Epoch 2/50
1906/1906 [==============================] - 2s 1ms/step - loss: 6.8953 - mse: 6.8953
Epoch 3/50
1906/1906 [==============================] - 2s 1ms/step - loss: 6.4439 - mse: 6.4439
Epoch 4/50
1906/1906 [==============================] - 2s 1ms/step - loss: 6.1090 - mse: 6.1090
Epoch 5/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.8487 - mse: 5.8487
Epoch 6/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.5637 - mse: 5.5637
Epoch 7/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.3508 - mse: 5.3508
Epoch 8/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.2272 - mse: 5.2272
Epoch 9/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.1328 - mse: 5.1328
Epoch 10/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.0525 - mse: 5.0525
Epoch 11/50
1906/1906

In [8]:
predictions = np.asarray(predictions) 
print(predictions.shape)

(28, 1, 30490)


In [16]:
predictions

array([[[0.59954804, 0.28839463, 0.48424977, ..., 0.9765709 ,
         0.96177197, 1.2463381 ]],

       [[1.0506003 , 0.25584638, 0.4186147 , ..., 0.84837294,
         1.5998545 , 1.3480214 ]],

       [[0.57119286, 0.2507323 , 0.5029304 , ..., 0.58642334,
         1.2414443 , 1.1173335 ]],

       ...,

       [[0.61766595, 0.63379896, 0.49924588, ..., 0.97382087,
         0.50332856, 2.368151  ]],

       [[1.286278  , 0.45445803, 0.48286524, ..., 1.1413032 ,
         1.7776657 , 1.451663  ]],

       [[1.3941919 , 0.3168364 , 0.28354362, ..., 0.35530216,
         1.2087046 , 1.7313887 ]]], dtype=float32)

In [17]:
# flatten each of predictions[i] 
pred_flattened = [] 
for i in range(28): 
    pred_flattened.append(predictions[i].flatten()) 

In [18]:
# checking if shape is flattened 
pred_flattened[0]

array([0.59954804, 0.28839463, 0.48424977, ..., 0.9765709 , 0.96177197,
       1.2463381 ], dtype=float32)

In [19]:
cols = [] 
for i in range(1,29): 
    cols.append("F"+str(i)) 
df = pd.DataFrame(pred_flattened)   
df = df.T # transpose 
df.columns = cols # rename column 
df 

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,0.599548,1.050600,0.571193,0.642809,0.780898,0.882063,0.991847,0.508978,0.684500,0.741584,...,0.532092,0.372174,1.270951,0.403613,0.482628,0.268334,0.637379,0.617666,1.286278,1.394192
1,0.288395,0.255846,0.250732,0.218349,0.464217,0.241705,0.562551,0.270284,0.130283,0.137436,...,0.002914,0.279062,0.336042,0.742035,-0.202891,0.298447,0.364905,0.633799,0.454458,0.316836
2,0.484250,0.418615,0.502930,0.392039,0.647606,0.296975,1.068568,0.842442,0.542551,0.653019,...,1.045637,0.581094,0.670881,0.324326,0.184632,0.896063,0.419366,0.499246,0.482865,0.283544
3,1.852196,1.398347,1.947028,2.557512,1.943759,1.565699,2.670388,2.791629,2.152992,2.125197,...,2.313348,2.196831,3.303088,1.990234,1.567045,3.581565,1.888692,3.071023,3.112190,4.552181
4,1.292570,0.896181,1.020372,1.111336,1.424503,1.091401,1.220740,0.670312,1.897139,1.648169,...,1.033693,0.479500,2.042438,1.707259,0.769617,0.309100,0.886835,2.025807,1.237649,0.986632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0.896780,0.544016,0.719606,0.788931,0.700758,0.759388,0.702191,0.453587,0.608242,0.987527,...,0.242652,1.428435,0.240424,0.599667,0.547424,0.519214,0.371383,0.387759,0.488768,0.214616
30486,-0.023480,0.017157,0.319203,0.249267,0.006660,0.138106,0.065137,-0.097202,0.088980,-0.097248,...,0.370858,-0.150680,0.326765,0.455847,0.068393,-0.932283,0.486196,0.552295,-0.377743,0.345861
30487,0.976571,0.848373,0.586423,0.928971,0.585786,0.369162,0.811884,0.716548,0.292051,0.606176,...,0.435023,0.387653,1.036312,0.331551,0.106291,0.304136,0.489319,0.973821,1.141303,0.355302
30488,0.961772,1.599854,1.241444,1.212791,0.871150,1.462639,1.100845,0.806203,1.297328,1.737480,...,0.583048,1.200651,1.092644,0.767820,0.755602,1.374979,0.766074,0.503329,1.777666,1.208705


In [20]:
validation_ids = df_train['id'].values
evaluation_ids = [i.replace('validation', 'evaluation') for i in validation_ids]
ids = np.concatenate([validation_ids, evaluation_ids])       
ids

array(['HOBBIES_1_001_CA_1_validation', 'HOBBIES_1_002_CA_1_validation',
       'HOBBIES_1_003_CA_1_validation', ...,
       'FOODS_3_825_WI_3_evaluation', 'FOODS_3_826_WI_3_evaluation',
       'FOODS_3_827_WI_3_evaluation'], dtype=object)

In [21]:
sub_df = pd.DataFrame(ids, columns=['id']) 
df = pd.concat([df]*2).reset_index(drop=True) 
sub_df = pd.concat([sub_df,df],axis=1) 
sub_df.to_csv('LSTM_128_append.csv',index=False) 

In [22]:
sub_df 

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.599548,1.050600,0.571193,0.642809,0.780898,0.882063,0.991847,0.508978,0.684500,...,0.532092,0.372174,1.270951,0.403613,0.482628,0.268334,0.637379,0.617666,1.286278,1.394192
1,HOBBIES_1_002_CA_1_validation,0.288395,0.255846,0.250732,0.218349,0.464217,0.241705,0.562551,0.270284,0.130283,...,0.002914,0.279062,0.336042,0.742035,-0.202891,0.298447,0.364905,0.633799,0.454458,0.316836
2,HOBBIES_1_003_CA_1_validation,0.484250,0.418615,0.502930,0.392039,0.647606,0.296975,1.068568,0.842442,0.542551,...,1.045637,0.581094,0.670881,0.324326,0.184632,0.896063,0.419366,0.499246,0.482865,0.283544
3,HOBBIES_1_004_CA_1_validation,1.852196,1.398347,1.947028,2.557512,1.943759,1.565699,2.670388,2.791629,2.152992,...,2.313348,2.196831,3.303088,1.990234,1.567045,3.581565,1.888692,3.071023,3.112190,4.552181
4,HOBBIES_1_005_CA_1_validation,1.292570,0.896181,1.020372,1.111336,1.424503,1.091401,1.220740,0.670312,1.897139,...,1.033693,0.479500,2.042438,1.707259,0.769617,0.309100,0.886835,2.025807,1.237649,0.986632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.896780,0.544016,0.719606,0.788931,0.700758,0.759388,0.702191,0.453587,0.608242,...,0.242652,1.428435,0.240424,0.599667,0.547424,0.519214,0.371383,0.387759,0.488768,0.214616
60976,FOODS_3_824_WI_3_evaluation,-0.023480,0.017157,0.319203,0.249267,0.006660,0.138106,0.065137,-0.097202,0.088980,...,0.370858,-0.150680,0.326765,0.455847,0.068393,-0.932283,0.486196,0.552295,-0.377743,0.345861
60977,FOODS_3_825_WI_3_evaluation,0.976571,0.848373,0.586423,0.928971,0.585786,0.369162,0.811884,0.716548,0.292051,...,0.435023,0.387653,1.036312,0.331551,0.106291,0.304136,0.489319,0.973821,1.141303,0.355302
60978,FOODS_3_826_WI_3_evaluation,0.961772,1.599854,1.241444,1.212791,0.871150,1.462639,1.100845,0.806203,1.297328,...,0.583048,1.200651,1.092644,0.767820,0.755602,1.374979,0.766074,0.503329,1.777666,1.208705


In [16]:
#model.compile(loss='mse',optimizer='adam',metrics=['mse'])
#model_checkpoint = keras.callbacks.ModelCheckpoint('./storage/Kaggle_M5_Luke/model_base2/model{epoch:02d}.h5',period=50)
#model.fit(x_train,y_train,epochs=1000,callbacks=[model_checkpoint],validation_data=(x_val,y_val))  
